# AI Assistant

In this notebook you'll make an AI assistant to help customers make the best decision about getting a new shoe from Cloudrunners Shoes. You will also take a short dive into practical token limitations for the model you are working with, and where you can focus your learning in the case you need to build more powerful AI agent systems.

## Helper Functions

### Amazon SageMaker
Use Amazon SageMaker Jumpstart to deploy a Llama2-70B model. 

## Setup

***

In [2]:
%pip install --upgrade --quiet sagemaker


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


***
You can continue with the default model or choose a different model: this notebook will run with the following model IDs :
- `meta-textgeneration-llama-2-7b-f`
- `meta-textgeneration-llama-2-13b-f`
- `meta-textgeneration-llama-2-70b-f`
***

In [3]:
model_id, model_version = "meta-textgeneration-llama-2-70b-f", "*"

## Deploy model

***
You can now deploy the model using SageMaker JumpStart.
***

In [5]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id)
predictor = model.deploy()

---------------------------------------!

In [6]:
endpoint_name = predictor.endpoint_name

import boto3, json
sm_client = boto3.client('sagemaker-runtime')

def transform_input(prompt: str, temperature) -> bytes:
    input_str = json.dumps({
        "inputs": [[{"role": "user", "content": prompt},]],
        "parameters": {"max_new_tokens": 512,"top_p": 0.9,"temperature": temperature}
    })
    return input_str.encode('utf-8')
    
def transform_output(output: bytes) -> str:
    response_json = json.loads(output['Body'].read().decode("utf-8"))
    return response_json[0]["generation"]["content"]

def generate(prompt, temperature = 0.6):
    response = sm_client.invoke_endpoint(
        EndpointName=endpoint_name, 
        CustomAttributes="accept_eula=true", 
        ContentType="application/json",
        Body=transform_input(prompt, temperature)
    )
    response_output = transform_output(response)
    return response_output

### Edit System Message

In [7]:
def prompt_with_system_message(prompt, system_message):
    prompt = f"""
    <s>[INST] <<SYS>>{system_message}<</SYS>>

    User: {prompt}
    Agent:[/INST]
    """
    return prompt

### Include One-to-many Shot Learning

In [8]:
def prompt_with_examples(prompt, system_message, examples=[]):
    
    # Start with the initial part of the prompt with system message
    full_prompt = f"<s>[INST] <<SYS>>{system_message}<</SYS>>\n"

    # Add each example to the prompt
    for user_msg, agent_response in examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

### LlamaChatbot Class

In [9]:
class LlamaChatbot:
    def __init__(self, system_message):
        self.system_message = system_message
        self.conversation_history = []  # list of tuples (user_msg, agent_response)

    def chat(self, user_msg):
        # Generate the prompt using the conversation history and the new user message
        prompt = prompt_with_examples(user_msg, self.system_message, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        # Clear conversation history
        self.conversation_history = []

## Data

### Cloudrunners Shoes Details

In [10]:
shoes = [
    {
        "model": "Sky Glider",
        "type": "Running", 
        "features": {
            "upper": "Mesh",
            "sole": "EVA foam",
            "lacing": "Quick-tie",
            "drop": "8mm",
            "color": "Blue with yellow accents"
        },
        "usps": ["Lightweight", "Responsive cushioning", "Seamless upper"],
        "price": 119.95,
        "internal_id": "SG123",
        "weight": "220g",
        "manufacturer_location": "Vietnam"
    },
    {   
        "model": "Trail Trekker",
        "type": "Hiking",
        "features": {
            "upper": "Synthetic leather",
            "sole": "Rubber lug", 
            "lacing": "Traditional",
            "drop": "12mm",
            "color": "Khaki green"
        },
        "usps": ["Rugged construction", "Super grippy sole", "Waterproof"], 
        "price": 129.99,
        "internal_id": "TT321",
        "weight": "340g",
        "manufacturer_location": "China"
    },
    {
        "model": "Marathon Master",
        "type": "Racing",
        "features": {
            "upper": "Mesh",
            "sole": "Carbon fiber plate",
            "lacing": "Speed laces",
            "drop": "6mm",
            "color": "Neon yellow and black"
        },
        "usps": ["Maximizes energy return", "Lightning fast", "Seamless comfort"],
        "price": 179.50, 
        "internal_id": "MM111",
        "weight": "180g",
        "manufacturer_location": "USA"
    }
]

## Shoes AI Assistant

In [11]:
system_message = """
You are a friendly chatbot knowledgeable about shoes. \
When asked about specific shoe models or features, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability.
"""

chatbot = LlamaChatbot(system_message)

In [12]:
print(chatbot.chat("Can you tell me about the latest models?"))

 Of course! There are many great new shoe models available in various styles and brands. Can you please provide me with some more information about what you're looking for? For example, are you interested in running shoes, basketball shoes, casual sneakers, or dress shoes? Additionally, do you have a specific brand or price range in mind? This will help me provide you with more tailored recommendations.


In [13]:
chatbot.reset()

## Cloudrunners Shoes AI Assistant

In [14]:
system_message = f"""
You are a friendly chatbot knowledgeable about these bicycles from Cloudrunners Shoes {shoes}. \
When asked about specific shoe models or features, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability.
"""

chatbot = LlamaChatbot(system_message)

In [15]:
print(chatbot.chat("Can you tell me about the latest models?"))

 Sure, I'd be happy to help! We currently have three latest models in our lineup: the Sky Glider, Trail Trekker, and Marathon Master.

The Sky Glider is a running shoe designed for lightweight performance and responsive cushioning. It features a mesh upper, EVA foam sole, and quick-tie lacing system. It's a great choice for runners who want a comfortable, fast ride.

The Trail Trekker is a hiking shoe built for rugged construction and super grippy performance. It has a synthetic leather upper, rubber lug sole, and traditional lacing system. It's perfect for hikers who need a reliable shoe that can handle tough terrain.

The Marathon Master is a racing shoe designed for maximizing energy return and speed. It features a mesh upper, carbon fiber plate, and speed laces. It's a great choice for runners who want to take their performance to the next level.

All of our shoes are made with high-quality materials and designed to provide the best performance and comfort for their intended use. I

In [16]:
print(chatbot.chat("How much do each of the models cost?"))

 Sure, here are the prices for each of the models:

1. Sky Glider: $119.95
2. Trail Trekker: $129.99
3. Marathon Master: $179.50

I hope that helps! Let me know if you have any other questions.


In [17]:
print(chatbot.chat("I'm torn between the Sky Glider and the Marathon Master, how should I decide which is best or me?"))

 Both the Sky Glider and the Marathon Master are great options, so it's understandable that you're having trouble deciding between them! Here are some factors to consider that may help you make a decision:

1. Running style: The Sky Glider is designed for lightweight performance and responsive cushioning, making it a great choice for runners who want a fast and comfortable ride. The Marathon Master, on the other hand, is designed for maximizing energy return and speed, making it a great choice for runners who want to take their performance to the next level. Think about your running style and what you value most in a shoe. Do you prioritize comfort and cushioning, or do you want a shoe that will help you run faster and more efficiently?
2. Terrain: The Sky Glider is a great choice for running on roads and smooth trails, while the Marathon Master is designed for running on a variety of surfaces, including trails and rough terrain. If you do a lot of trail running, the Marathon Master's 

In [18]:
print(chatbot.chat("I'm torn between the Marathon Master and the Trail Trekker, how should I decide which is best or me?"))

 Both the Marathon Master and the Trail Trekker are great options, so it's understandable that you're having trouble deciding between them! Here are some factors to consider that may help you make a decision:

1. Running style: The Marathon Master is designed for maximizing energy return and speed, making it a great choice for runners who want to take their performance to the next level. It has a responsive midsole and a lightweight, breathable upper. The Trail Trekker, on the other hand, is designed for trail running and has a more rugged construction with a grippy outsole and a waterproof membrane. It's a great choice for runners who want a shoe that can handle rough terrain. Think about your running style and what you value most in a shoe. Do you prioritize speed and performance, or do you need a shoe that can handle trails and rough terrain?
2. Terrain: The Marathon Master is designed for running on roads and smooth trails, while the Trail Trekker is designed for running on trails 

In [19]:
chatbot.reset()